In [2]:
from __future__ import print_function
import xml.etree.ElementTree as ET
import nltk
from nltk.tokenize import WhitespaceTokenizer
from nltk.corpus import stopwords
import os

nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ggebbia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ggebbia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
def tokenize(sentence, span_generator):
    return [(sentence[span[0]: span[1]], span[0], span[1]-1) for span in span_generator]

In [21]:
def extract_entities(tokenized_list):
    entities_list = []
    last_type = None
    word_stripped = False
    list_length = len(tokenized_list)-1
    for i, token in enumerate(tokenized_list):
        word, offset_from, offset_to = token
        d = {}
        
        # remove . from last word 
        if index == list_length and word.endswith("."):
            word = word[:-1]
        # strip , and : from word
        elif (word.endswith(":") or word.endswith(",")):
            word = word[:-1]
            word_stripped = True
        
        
        if (word[0].isupper() and offset_from != 0) or (word.isupper()) or (word.lower()=="spermine"):
            if last_type != None:
                prev_word, prev_offset_from, _ = tokenized_list[i-1]
                entities_list.pop(-1) 
                d["name"] = prev_word + " acid"
                d["type"] = "drug"
                d["offset"] = "{}-{}".format(prev_offset_from, offset_to)
            else:
                d["name"] = word
                d["type"] = "drug" # Posar pes per fer probabilitat 2/3 si es drug, 1/3 si es brand
                d["offset"] = "{}-{}".format(offset_from, offset_to)             
            last_type = "drug"                  
        
        #TODO: si la paraula seguent tambe te el mateix tipus, merge
        elif word.lower() == "acid":
            prev_word, prev_offset_from, _ = tokenized_list[i-1]
            
            # Remove drug or brand if it was added since the next word is acid
            if last_type != None:
                entitites_list.pop(-1)            
            d["name"] = prev_word + " acid"
            d["type"] = "drug"
            d["offset"] = "{}-{}".format(prev_offset_from, offset_to)
            last_type = "drug"
    
        else:
            last_type = None
        
        # If this word was ending with , or : next word not mergerd with this one
        if word_stripped:
            last_type = None
            word_stripped = False
            
        if d.keys(): entities_list.append(d)
    return entities_list

In [5]:
def output_entities(sid, entities, outputfile):
    for ent in entities:
        line = '|'.join([sid, ent["offset"], ent["name"], ent["type"]])
        outputfile.write(line + "\n")

In [6]:
def evaluate(inputdir, outputfile):
    os.system("java -jar eval/evaluateNER.jar "+ inputdir + " " + outputfile)

In [10]:
### VARIABLES
inputdir = "./data/Devel"
outputfilename="./output.txt"
outputfile = open(outputfilename, "w")

for filename in os.listdir(inputdir):
    
    file_path = os.path.join(inputdir, filename)  
    tree = ET.parse(file_path)
    root = tree.getroot()

    for child in root:
        (sid, text) = (child.attrib["id"], child.attrib["text"])
        span_generator = WhitespaceTokenizer().span_tokenize(text)
        tokenized_list = tokenize(text, span_generator)
        entities = extract_entities(tokenized_list)
        output_entities(sid, entities, outputfile)

evaluate(inputdir, outputfilename)
outputfile.close()